In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"
midopluskey_path = KEY_PATH + "midoplus.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [7]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [8]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [9]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path) ## 개인계정
# all_shop_df = get_dataframe_from_bigquery('DATA_WAREHOUSE', 'g2b_data', midopluskey_path) ## 회사계정

In [10]:
years = today[:-4]
all_shop_df = all_shop_df[all_shop_df['납품요구접수일자'].str.split('-').str[0]==years].reset_index(drop=True)
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [11]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [12]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
# all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [13]:
# 수요기관 필터링
dist_nm = pd.Series(all_shop_df_fin['수요기관지역명'].str.split().str[1].dropna().unique())
dist_nm = np.where(dist_nm.apply(len)<=2,dist_nm,dist_nm.str[:-1])

#### 교육청 예산서

In [14]:
# 빅쿼리에서 불러오기(개인계정)
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

In [36]:
# # 빅쿼리에서 불러오기(회사계정)
# edu_budget_df = get_dataframe_from_bigquery('DATA_MARTS','list_up_edu_budget_data',midopluskey_path) ## 회사계정
# edu_budget_df = edu_budget_df.rename(columns={'도광역시':'시도'})

In [37]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
# edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [38]:
# 교육청 기준 지역명, 교육청명 종합쇼핑몰 형식과 통일
# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
edu_budget_df['시도_re'] = edu_budget_df['시도'].replace(replacements)
edu_budget_df['시도_re'] = edu_budget_df['시도_re'].str[:2]

edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [39]:
# 학교 관련 키워드 및 데이터 추출
edu_budget_df['교육청_키워드'] = edu_budget_df['과업명'].apply(lambda x: ''.join(re.findall('\\w+초|\\w+초등학교|\\w+중|\\w+중학교|\\w+고|\\w+고등학교|\\w+학교', x)))

edu_budget_df['교육청_키워드2'] = edu_budget_df['교육청_키워드'].str.split('초|중|고|학교').str[0] ## 학교명 추출

# 일반고 외 학교명 전처리
replacements = {
    '여자': '여',
    '공업': '공',
    '상업': '상',
    '농업': '농',
    '과학': '과'
}
edu_budget_df['교육청_키워드2'] = edu_budget_df['교육청_키워드2'].replace(replacements, regex=True)

In [40]:
# 학교명 등 필요키워드 추출
edu_kwd_temp = edu_budget_df['교육청_키워드2'].dropna().unique()
edu_kwd = edu_kwd_temp[edu_kwd_temp!='']

In [41]:
# 해당 필요키워드 추출
edu_budget_df['키워드'] = edu_budget_df['교육청_키워드2'].apply(lambda x: ''.join(re.findall('|'.join(edu_kwd), x))) ## 학교명
edu_budget_df['키워드2'] = edu_budget_df['교육청_키워드'].apply(lambda x: ''.join(re.findall('초|중|고', x)))

#### 교육청 종합쇼핑몰 전처리

In [42]:
all_shop_df_fin['키워드'] = all_shop_df_fin['납품요구건명'].apply(lambda x: ''.join(re.findall('|'.join(edu_kwd), x)))
all_shop_df_fin['키워드2'] = all_shop_df_fin['납품요구건명'].apply(lambda x: ''.join(re.findall('초|중|고', x)))

In [43]:
# 종합쇼핑몰 기준 지역명 형식과 통일
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명_split1'].replace(replacements)
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명_split1'].str[:2]

In [44]:
# 지역명 추출2
all_shop_df_fin['수요기관지역명_split2'] = all_shop_df_fin['수요기관지역명'].str.split().str[1]

all_shop_df_fin['수요기관지역명_split2'] = np.where(all_shop_df_fin['수요기관지역명_split2'].isnull(),
                                                  all_shop_df_fin['수요기관지역명_split1'],
                                                  all_shop_df_fin['수요기관지역명_split2'])

all_shop_df_fin['수요기관지역명_split2'] = np.where(all_shop_df_fin['수요기관지역명_split2'].apply(len)<=2,
                                                  all_shop_df_fin['수요기관지역명_split2'],
                                                  all_shop_df_fin['수요기관지역명_split2'].str[:-1])

dist_sig_pat = '|'.join(all_shop_df_fin['수요기관지역명_split2'].dropna().unique())

In [45]:
# 교육청 지역명 추출
edu_budget_df['시군구_re2'] = edu_budget_df['시군구_re'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
edu_budget_df['시군구_re2'] = edu_budget_df['시군구_re2'].str.strip()

In [46]:
# 학교 키워드 기준 병합
mapping_merge_edu_df_fin = pd.merge(edu_budget_df[['시도', '시군구', '과업명', '시도_re', '시군구_re', '시군구_re2', '키워드','키워드2']],
                                all_shop_df_fin[['수요기관명', '납품요구건명', '업체명', '수요기관지역명', '납품요구지청명', '수요기관지역명_split1', '수요기관지역명_split2', '키워드','키워드2']],
                                how='left', on=['키워드','키워드2'])
mapping_merge_edu_df_fin = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['키워드']!=''].drop_duplicates().reset_index(drop=True)
mapping_merge_edu_df_fin = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['납품요구건명'].notnull()].drop_duplicates().reset_index(drop=True)

In [47]:
# 지역 일치 확인
mapping_merge_edu_df_final = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['시도_re']==mapping_merge_edu_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_edu_df_final

,시도,시군구,과업명,시도_re,시군구_re,시군구_re2,키워드,키워드2,수요기관명,납품요구건명,업체명,수요기관지역명,납품요구지청명,수요기관지역명_split1,수요기관지역명_split2
0,강원특별자치도,강원특별자치도교육청,봉의고등학교 인조잔디 운동장 및 우레탄조성 테니스장 조성,강원,강원특별자치도교육청,,봉의,고,강원특별자치도교육청 강원특별자치도춘천교육지원청,봉의고 인조잔디운동장 및 테니스장 조성공사 관급자재(인조잔디) 구입,지앤지텍 주식회사,강원도 춘천시,강원지방조달청,강원,춘천
1,강원특별자치도,춘천교육지원청,부안초등학교 인조잔디 운동장 및 우레탄조성,강원,강원특별자치도춘천,춘천,부안,초,강원특별자치도교육청 강원특별자치도춘천교육지원청,부안초 인조잔디운동장 조성공사 관급자재(인조잔디) 구입,(주)대건씨앤엘,강원도 춘천시,강원지방조달청,강원,춘천
2,강원특별자치도,춘천교육지원청,춘천초등학교 다목적구장조성,강원,강원특별자치도춘천,춘천,춘천,초,강원특별자치도교육청 강원특별자치도춘천교육지원청,춘천초 다목적구장 조성공사 관급자재(인조잔디) 구입,코니터프주식회사,강원도 춘천시,강원지방조달청,강원,춘천
3,강원특별자치도,춘천교육지원청,춘천초등학교 다목적구장조성,강원,강원특별자치도춘천,춘천,춘천,초,강원특별자치도교육청 강원특별자치도춘천교육지원청,춘천초 다목적구장 조성공사 관급자재(인조잔디) 구입,한국체육조경주식회사,강원도 춘천시,강원지방조달청,강원,춘천
4,경기도,김포교육지원청,신양중학교 친환경 운동장 조성 사업,경기,경기도김포,김포,신양,중,경기도교육청 경기도김포교육청 신양중학교,2024년 신양중 인조잔디 운동장 조성물품 구입(인조잔디),주식회사 이원,경기도 김포시,인천지방조달청,경기,김포
5,경기도,동두천양주교육지원청,보산초등학교 친환경 운동장 조성 사업,경기,경기도동두천양주,동두천 양주,보산,초,경기도동두천교육청 보산초등학교,보산초 친환경 운동장 조성 인조잔디 구입,주식회사 플랜에이,경기도 동두천시,서울지방조달청,경기,동두천
6,광주광역시,광주광역시교육청,광주석산고등학교(인조잔디) (학교회계전출금),광주,광주광역시교육청,광주,광주석산,고,광주광역시교육청 광주석산고등학교,광주석산고 관급자재(인조잔디) 구매,주식회사 유니스포텍,광주광역시 남구,광주지방조달청,광주,남구
7,부산광역시,부산광역시교육청,배정고등학교 인조잔디 (재) 조성,부산,부산광역시교육청,,배정,고,부산광역시교육청 배정미래고등학교,배정미래고등학교 운동장 조성공사 관급자재,한국체육조경주식회사,부산광역시 남구,부산지방조달청,부산,남구
8,부산광역시,서부교육지원청,건국중학교 인조잔디 (재) 조성,부산,부산광역시서부,,건국,중,부산광역시서부교육청 건국중학교,건국중학교 인조잔디 운동장 조성공사 관급(인조잔디) 요청,주식회사 네오필드,부산광역시 사하구,부산지방조달청,부산,사하
9,강원특별자치도,강원특별자치도교육청,강릉고등학교 야구장 시설개선,강원,강원특별자치도교육청,,강릉,고,강원특별자치도교육청 강원특별자치도강릉교육지원청,강릉제일고 축구장 인조잔디 교체공사 관급자재(인조잔디) 구입,주식회사 그린에셀,강원특별자치도 강릉시,강원지방조달청,강원,강릉


In [48]:
# # 오매칭 확인 및 제거
# # delete_idx = [0,2,3]
# delete_idx = [7,9,14]
# mapping_merge_edu_df_final.drop(delete_idx, inplace=True)

In [49]:
edu_budget_df_final = edu_budget_df[~edu_budget_df['과업명'].isin(mapping_merge_edu_df_final['과업명'])].reset_index(drop=True)

In [50]:
edu_budget_df_final

,시도,시군구,구분,과업명,삭제,금액,면적,예산집행,과업명_re,시도_re,시군구_re,교육청_키워드,교육청_키워드2,키워드,키워드2,시군구_re2
0,강원특별자치도,강원특별자치도교육청,공립,북평여자고등학교 힐링운동장환경조성,False,330897000.0,,,북평여자고등학교 힐링운동장환경조성,강원,강원특별자치도교육청,북평여자고등학교,북평여,북평여,고,
1,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,False,51700000.0,,,동해광희고등학교 인조잔디운동장 보수,강원,강원특별자치도교육청,동해광희고등학교,동해광희,동해광희,고,
2,강원특별자치도,교육부,국립,강원대학교사범대학부설고등학교 인조잔디운동장조성 및 환경개선,False,2442350000.0,,,강원대학교사범대학부설고등학교 인조잔디운동장조성 및 환경개선,강원,교육부,강원대학교사범대학부설고등학교,강원대,강원대,고,
3,강원특별자치도,춘천교육지원청,공립,봉의초등학교 테니스장교체,False,613290000.0,,,봉의초등학교 테니스장교체,강원,강원특별자치도춘천,봉의초등학교,봉의,봉의,초,춘천
4,경기도,경기도교육청,공립,향동고등학교 친환경 운동장 조성 사업,False,,,,향동고등학교 친환경 운동장 조성 사업,경기,경기도교육청,향동고등학교,향동,향동,고,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,충청남도,논산계룡교육지원청,공립,강경여자중학교 인조잔디 교체,False,150144000.0,5770.0,교육청,강경여자중학교 인조잔디 교체,충남,충청남도논산계룡,강경여자중,강경여,강경여,중,논산 계룡
157,충청남도,부여교육지원청,공립,부여여자중학교 유해운동장 교체,False,132948000.0,2720.0,교육청,부여여자중학교 유해운동장 교체,충남,충청남도부여,부여여자중,부여여,부여여,중,부여
158,충청남도,예산교육지원청,공립,예산여자중학교 유해운동장 교체,False,259105000.0,2900.0,교육청,예산여자중학교 유해운동장 교체,충남,충청남도예산,예산여자중,예산여,예산여,중,예산
159,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,False,602871000.0,3500.0,교육청,서경중학교 운동장 보수 및 바닥교체,충북,충청북도청주,서경중,서경,서경,중,청주


In [29]:
# 개인계정
save_dataframe_to_bigquery(edu_budget_df_final,'edu','edu_budget_listup',bigquerykey_path)
# edu_budget_df_final.to_csv('C:\py_src\midoproject\result/filtered_edu_df_final.csv',encoding='utf-8-sig',index=False)

Data inserted into table edu_budget_listup successfully.
